### RL and Advanced DL: Домашнее задание 1

Первое ДЗ связано с обучением с подкреплением, и оно придумано для ситуации, когда нейронные сети ещё не нужны, и пространство состояний в целом достаточно маленькое, чтобы можно было обучить хорошую стратегию методами TD-обучения или другими методами обучения с подкреплением.

### Часть I

Мы будем обучаться играть в очень простую, но знаменитую и популярную игру: блекджек. 
 
Правила блекджека достаточно просты; давайте начнём с самой базовой версии, которая реализована в OpenAI Gym:<br>
➢	численные значения карт равны от 2 до 10 для карт от двойки до десятки, 10 для валетов, дам и королей;<br>
➢	туз считается за 11 очков, если общая сумма карт на руке при этом не превосходит 21 (по-английски в этом случае говорят, что на руке есть usable ace), и за 1 очко, если превосходит;<br>
➢	игроку раздаются две карты, дилеру — одна в открытую и одна в закрытую;<br>
➢	игрок может совершать одно из двух действий:<br>
○	hit  — взять ещё одну карту;<br>
○	stand — не брать больше карт;<br>
➢	если сумма очков у игрока на руках больше 21, он проигрывает (bust);<br>
➢	если игрок выбирает stand с суммой не больше 21, дилер добирает карты, пока сумма карт в его руке меньше 17;<br>
➢	после этого игрок выигрывает, если дилер либо превышает 21, либо получает сумму очков меньше, чем сумма очков у игрока; при равенстве очков объявляется ничья (ставка возвращается);<br>
➢	в исходных правилах есть ещё дополнительный бонус за natural blackjack: если игрок набирает 21 очко с раздачи, двумя картами, он выигрывает не +1, а +1.5 (полторы ставки).<br>
Именно этот простейший вариант блекджека реализован в OpenAI Gym:<br>
https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py 
1.	Рассмотрим очень простую стратегию: говорить stand, если у нас на руках комбинация в 19, 20 или 21 очко, во всех остальных случаях говорить hit. Используйте методы Монте-Карло, чтобы оценить выигрыш от этой стратегии.<br>
2.	Реализуйте метод обучения с подкреплением без модели (можно Q-обучение, но рекомендую попробовать и другие, например Monte Carlo control) для обучения стратегии в блекджеке, используя окружение Blackjack-v0 из OpenAI Gym.<br>
3.	Сколько выигрывает казино у вашей стратегии? Нарисуйте графики среднего дохода вашего метода (усреднённого по крайней мере по 100000 раздач, а лучше больше) по ходу обучения. Попробуйте подобрать оптимальные гиперпараметры.<br>


In [4]:
# import blackjack

In [20]:
import gym
from gym import envs
import numpy as np

In [19]:
# print(envs.registry.all())

In [10]:
env=gym.make('Blackjack-v1', natural=True)

In [21]:
def simple_strategy_episode(env):
    """
    observations:
        score_player, score_dealer, usable_ace [natural blackjack]
    actions: 0 - stand, 1 - hit
    """
    obs, actions, rewards = [], [], []
    observation = env.reset()
    done = False
    while not done:
        score_player, score_dealer, usable_ace = observation
        action = 1 if score_player < 19 else 0
        actions.append(action)
        obs.append(observation)
        observation, reward, done, _ = env.step(action)
        rewards.append(reward)
    return obs, actions, rewards

In [23]:
#Single example
obs, actions, rewards = simple_strategy_episode(env)
print(f"observations[score_player, score_dealer, usable_ace]={obs}, actions={actions}, rewards={rewards}" )

observations[score_player, score_dealer, usable_ace]=[(9, 1, False), (18, 1, False), (21, 1, False)], actions=[1, 1, 0], rewards=[0.0, 0.0, 1.0]


In [43]:
num_of_episodes = int(1e5)
simple_strategy_reward = 0
for i in range(num_of_episodes): 
    _, _, rewards = simple_strategy_episode(env)
    simple_strategy_reward = (simple_strategy_reward * (i + 1) + rewards[-1]) / ((i + 1) + 1)

In [44]:
print(f"Reward for simple strategy is {simple_strategy_reward}")

Reward for simple strategy is -0.19606803931960404


In [ ]:
env.close()

#### Reward for simple strategy is -0.196

Реализуйте метод обучения с подкреплением без модели (можно Q-обучение, но рекомендую попробовать и другие, например Monte Carlo control) для обучения стратегии в блекджеке, используя окружение Blackjack-v0 из OpenAI Gym.